In [ ]:
import numpy as np
import cv2
import os
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.regularizers import l2
from keras.callbacks import EarlyStopping
from tensorflow.keras.layers import AveragePooling2D
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, multilabel_confusion_matrix, roc_auc_score

In [ ]:
training_data = []
DATADIR = '/Users/saraali/NBM_DeepLearning_Gamma/Project/Data_thr_aug/'
CATEGORIES = ['abnormal_thr','History_of_MI_thr','myocardial_Infarction_thr','normal_thr']

def create_train_data():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num= CATEGORIES.index(category)
        for img_name in os.listdir(path):
            img_array = cv2.imread(os.path.join(path,img_name), cv2.IMREAD_GRAYSCALE)
            img_array = cv2.resize(img_array, (420,250))
            training_data.append([img_array,class_num])


create_train_data()

#shuffle the data 

random.shuffle(training_data)

len(training_data)

X =[]
y =[]

for features,label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1,420,250,1).astype(np.int64)
y = np.array(y)

# scale the data by the max pixel vlaue 
X = X/255

X_train, X_test, y_train, y_test = (train_test_split(X, y, 
                                                     test_size = .2, random_state = 42))

y_train = to_categorical(y_train)

In [ ]:
earlystop = EarlyStopping(monitor = 'val_accuracy',
                          min_delta = 0,
                          patience = 1,
                          mode='max',
                          verbose = 1,
                          restore_best_weights = True)

In [ ]:
model = Sequential()

model.add(InputLayer(input_shape=X_train.shape[1:]))

model.add(Conv2D(35, (3,3), strides=(1, 1),padding='same',kernel_regularizer=l2(0.01)))
model.add(Activation('relu'))

model.add(Conv2D(20, (3,3), strides=(1, 1),padding='same',kernel_regularizer=l2(0.1)))
model.add(Activation('relu'))

model.add(Conv2D(15, (3,3), strides=(1, 1),padding='same', kernel_regularizer=l2(10)))
model.add(Activation('relu'))

model.add(AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding="same"))
model.add(MaxPooling2D(pool_size = (1,1),strides=(2, 2)))

model.add(Flatten())
          
model.add(Dense(15, kernel_regularizer=l2(10)))
model.add(Activation('relu'))
model.add(Dense(4)) 
model.add(Activation('softmax'))

model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 1, verbose = 1, validation_data = (X_test, to_categorical(y_test)), callbacks = earlystop)

# Metrics
        1- Arrhytmia
        2- History of MI 
        3- MI
        4- Normal

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

In [ ]:
accuracy_score(y_pred,to_categorical(y_test))

In [ ]:
precision_score(y_pred,to_categorical(y_test),average=None) 

In [ ]:
recall_score(y_pred,to_categorical(y_test),average=None)

In [ ]:
f1_score(y_pred,to_categorical(y_test),average=None)

In [ ]:
roc_auc_score(y_pred,to_categorical(y_test), average=None)

In [ ]:
multilabel_confusion_matrix(y_pred,to_categorical(y_test))